<a href="https://colab.research.google.com/github/lcurbelo/DVWA/blob/master/Trading_Mutual_Funds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install yfinance pandas

In [ ]:
# ===============================
# IMPORTS
# ===============================
import pandas as pd
import yfinance as yf
import time

# ===============================
# CONFIG — ALL TICKERS
# ===============================
MARKET = {"SPY": "SPY", "QQQ": "QQQ", "VIX": "^VIX"}

BASE_BUYS = {"FBGRX": 25, "FCNTX": 25, "FXAIX": 10}

US_ENGINE = ["FSELX", "FOCPX", "FSPTX", "FBMPX"]
INTERNATIONAL = "FHKCX"
DEFENSE = "FSDAX"
GOLD = "FIJDX"
OPPORTUNISTIC = ["FSPCX", "FSHOX"]

# ===============================
# PARAMETERS
# ===============================
TD_1M = 21
TD_3M = 63
TD_12M = 252
TD_200 = 200

MODE = "ticket"          # "ticket" or "full"
REQUIRE_FRESH = False    # set True after NAVs post
RETRIES = 3

# ===============================
# HELPERS
# ===============================
def pct(a, b):
    return (a / b - 1) * 100

def fetch_close(ticker):
    last_err = None
    for attempt in range(RETRIES):
        try:
            df = yf.download(
                ticker,
                period="max",
                interval="1d",
                auto_adjust=True,
                progress=False,
                threads=False,
                group_by="column"
            )
            if df is None or df.empty:
                raise ValueError("Empty Yahoo response")

            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [c[0] for c in df.columns]

            if "Close" not in df.columns:
                raise ValueError("Close not found")

            close = df["Close"]
            if isinstance(close, pd.DataFrame):
                close = close.iloc[:, 0]

            close = close.dropna()
            close.index = pd.to_datetime(close.index, errors="coerce")
            close = close.dropna().sort_index()

            if close.empty:
                raise ValueError("Empty close series")

            return close
        except Exception as e:
            last_err = e
            time.sleep(0.6 * (attempt + 1))
    raise RuntimeError(f"{ticker} failed: {last_err}")

def metrics(close):
    n = len(close)
    asof = close.index[-1].date().isoformat()
    last = float(close.iat[-1])

    dma200 = close.rolling(TD_200).mean().iat[-1]
    dma200 = float(dma200) if pd.notna(dma200) else None
    trend = "ABOVE" if dma200 and last > dma200 else "BELOW"

    r1m = pct(last, close.iat[-1 - TD_1M]) if n > TD_1M else None
    r3m = pct(last, close.iat[-1 - TD_3M]) if n > TD_3M else None

    high12 = close.iloc[-TD_12M:].max() if n > TD_12M else None
    drawdown = pct(last, high12) if high12 is not None else None

    return {
        "asof": asof,
        "last": last,
        "dma200": dma200,
        "trend": trend,
        "r1m": r1m,
        "r3m": r3m,
        "drawdown": drawdown
    }

def yesno(x): return "YES" if x else "NO"

# ===============================
# MAIN RUN
# ===============================
def run():
    spy = metrics(fetch_close("SPY"))
    qqq = metrics(fetch_close("QQQ"))

    risk_on = spy["trend"] == "ABOVE" and qqq["trend"] == "ABOVE"

    # VIX
    try:
        vix = fetch_close("^VIX")
        vix_last = float(vix.iat[-1])
        vix_30 = float(vix.rolling(30).mean().iat[-1])
        vix_elev = vix_last > vix_30
    except:
        vix_elev = None

    us = {t: metrics(fetch_close(t)) for t in US_ENGINE}
    intl = metrics(fetch_close(INTERNATIONAL))
    defense = metrics(fetch_close(DEFENSE))
    gold = metrics(fetch_close(GOLD))
    opp = {t: metrics(fetch_close(t)) for t in OPPORTUNISTIC}

    if REQUIRE_FRESH:
        target = spy["asof"]
        stale = [t for t,m in {**us, INTERNATIONAL:intl, DEFENSE:defense, GOLD:gold}.items()
                 if m["asof"] != target]
        if stale:
            raise RuntimeError(f"STALE DATA: {stale}")

    us_ok = {t: risk_on and us[t]["r1m"] > 0 and us[t]["trend"] == "ABOVE" for t in US_ENGINE}

    intl_ok = risk_on and intl["r1m"] > 0 and intl["r3m"] > 0 and intl["trend"] == "ABOVE"
    def_ok = risk_on and defense["r1m"] > 0 and defense["trend"] == "ABOVE"
    gold_ok = spy["trend"] == "ABOVE" and gold["r1m"] > 0 and gold["trend"] == "ABOVE" and (vix_elev if vix_elev is not None else True)

    opp_ok = [t for t in OPPORTUNISTIC if opp[t]["drawdown"] is not None and opp[t]["drawdown"] <= -20]

    # Engine
    if not risk_on:
        engine = "FCNTX"
    else:
        best_us = max((t for t in US_ENGINE if us_ok[t]), key=lambda t: us[t]["r1m"], default=None)
        if intl_ok and best_us and intl["r1m"] - us[best_us]["r1m"] >= 5:
            engine = INTERNATIONAL
        else:
            engine = best_us if best_us else "FCNTX"

    ticket = dict(BASE_BUYS)
    ticket[engine] = ticket.get(engine, 0) + 80
    ticket["FXAIX"] += 200 - sum(ticket.values())

    sleeves = []
    if def_ok: sleeves.append(DEFENSE)
    if gold_ok: sleeves.append(GOLD)
    if opp_ok: sleeves.append(opp_ok[0])
    sleeves = sleeves[:2]

    for s in sleeves:
        ticket["FXAIX"] -= 10
        ticket[s] = ticket.get(s, 0) + 10

    if sum(ticket.values()) != 200:
        raise RuntimeError("Ticket total mismatch")

    if MODE == "ticket":
        for k in sorted(ticket):
            print(f"{k}: ${ticket[k]}")
    else:
        print("PART 1 — MARKET REGIME")
        print(f"SPY: {spy['trend']} | QQQ: {qqq['trend']} | Risk-On: {yesno(risk_on)}")
        if vix_elev is not None:
            print(f"VIX Elevated: {yesno(vix_elev)}")
        print("\nPART 9 — MONDAY TRADE TICKET")
        for k in sorted(ticket):
            print(f"{k}: ${ticket[k]}")

In [ ]:
if REQUIRE_FRESH:
    target = spy["asof"]
    required = {**us, INTERNATIONAL: intl, DEFENSE: defense, GOLD: gold}
    stale = [(t, required[t]["asof"]) for t in required if required[t]["asof"] != target]
    if stale:
        print(f"SPY as-of (target): {target}")
        print("STALE tickers (ticker, as-of):")
        for t, a in stale:
            print(f"  {t}: {a}")
        raise RuntimeError("STALE DATA (one or more funds not updated to target as-of)")

In [ ]:
def friday_run(mode="full"):
    global MODE, REQUIRE_FRESH
    MODE = mode

    REQUIRE_FRESH = True
    try:
        print("Attempt 1: REQUIRE_FRESH=True")
        run()
        return
    except Exception as e:
        msg = str(e).upper()
        if "STALE" not in msg:
            print("Non-stale error:", repr(e))
            raise

    print("\nNAVs not fully posted yet.")
    print("Attempt 2: REQUIRE_FRESH=False (preview)")
    REQUIRE_FRESH = False
    run()

In [ ]:
friday_run("full")

In [ ]:
# ===============================
# INSTITUTIONAL ROTATION ELIGIBILITY & MONDAY TRADE TICKET (DATA ONLY)
# Data Sources: Yahoo Finance (via yfinance). Fidelity tickers used directly.
# Paste into ONE Colab cell, run once. Then run RUN_FULL() or RUN_TICKET().
# ===============================

import pandas as pd
import yfinance as yf
import time

# -------------------------------
# CONFIG
# -------------------------------
MARKET = {"SPY": "SPY", "QQQ": "QQQ", "VIX": "^VIX"}

US_ENGINE = ["FSELX", "FOCPX", "FSPTX", "FBMPX"]
INTERNATIONAL = "FHKCX"
DEFENSE = "FSDAX"
GOLD = "FIJDX"
OPPORTUNISTIC = ["FSPCX", "FSHOX"]

BASE_BUYS = {"FBGRX": 25, "FCNTX": 25, "FXAIX": 10}
BUDGET_TOTAL = 200
ENGINE_BUDGET = 80
SLEEVE_AMOUNT = 10
MAX_SLEEVES = 2

# Trading-day approximations
TD_1M = 21
TD_3M = 63
TD_12M = 252
TD_200 = 200

# Runtime controls
RETRIES = 4
SLEEP_BETWEEN_RETRIES_SEC = 0.8

# Freshness controls (for Friday 4:15pm ET+)
REQUIRE_FRESH = True           # If True, fail if required funds not updated to SPY as-of date
FRESH_RETRY_MINUTES = 0        # If >0, will retry fresh check loop (e.g., 90) before failing
FRESH_RETRY_EVERY_MINUTES = 10 # Retry interval for freshness loop

# -------------------------------
# CORE DATA FETCH + METRICS
# -------------------------------
def _pct(a, b):
    return (a / b - 1.0) * 100.0

def fetch_close_series(ticker: str) -> pd.Series:
    """Fetch adjusted daily close series from Yahoo via yfinance. Returns a 1-D Series."""
    last_err = None
    for attempt in range(1, RETRIES + 1):
        try:
            df = yf.download(
                ticker,
                period="max",
                interval="1d",
                auto_adjust=True,
                progress=False,
                threads=False,
                group_by="column",
            )
            if df is None or df.empty:
                raise ValueError("Empty Yahoo response")

            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [c[0] for c in df.columns]

            if "Close" not in df.columns:
                raise ValueError(f"Missing Close column. Columns: {list(df.columns)[:10]}")

            close = df["Close"]
            if isinstance(close, pd.DataFrame):
                close = close.iloc[:, 0]

            close = close.dropna()
            close.index = pd.to_datetime(close.index, errors="coerce")
            close = close.dropna().sort_index()

            if close.empty:
                raise ValueError("Close series empty after cleaning")

            return close

        except Exception as e:
            last_err = e
            time.sleep(SLEEP_BETWEEN_RETRIES_SEC * attempt)

    raise RuntimeError(f"[{ticker}] Yahoo fetch failed after {RETRIES} tries: {last_err}")

def compute_metrics(close: pd.Series) -> dict:
    """Compute last, 200DMA trend, 1M/3M returns, 12M high and drawdown, as-of date."""
    close = close.dropna().sort_index()
    n = len(close)
    if n < 10:
        raise ValueError("Too little data")

    asof = close.index[-1].date().isoformat()
    last = float(close.iat[-1])

    dma200 = close.rolling(TD_200).mean().iat[-1]
    dma200 = float(dma200) if pd.notna(dma200) else None
    trend = None
    if dma200 is not None:
        trend = "ABOVE" if last > dma200 else "BELOW"

    r1m = None
    if n > TD_1M:
        r1m = _pct(last, float(close.iat[-1 - TD_1M]))

    r3m = None
    if n > TD_3M:
        r3m = _pct(last, float(close.iat[-1 - TD_3M]))

    high12 = None
    drawdown = None
    if n > TD_12M:
        window = close.iloc[-TD_12M:]
        high12 = float(window.max())
        drawdown = _pct(last, high12) if high12 is not None else None  # negative if below high

    return {
        "asof": asof,
        "last": last,
        "dma200": dma200,
        "trend": trend,
        "r1m": r1m,
        "r3m": r3m,
        "high12": high12,
        "drawdown": drawdown,
        "source": "Yahoo Finance (yfinance)",
    }

def _yesno(x: bool) -> str:
    return "YES" if x else "NO"

def _require(name: str, val, ticker: str):
    if val is None:
        raise RuntimeError(f"Missing {name} for {ticker} (insufficient Yahoo history)")
    return val

# -------------------------------
# PARTS 1–6: DATA GATHERING
# -------------------------------
def gather_all_data() -> dict:
    # Market
    spy = compute_metrics(fetch_close_series(MARKET["SPY"]))
    qqq = compute_metrics(fetch_close_series(MARKET["QQQ"]))

    # Optional VIX
    vix = None
    try:
        vix = compute_metrics(fetch_close_series(MARKET["VIX"]))
        # VIX 30d average computed from closes (calendar days, trading days approximation)
        vix_series = fetch_close_series(MARKET["VIX"])
        if len(vix_series) >= 31:
            vix_last = float(vix_series.iat[-1])
            vix_30 = float(vix_series.rolling(30).mean().iat[-1])
            vix["vix_last"] = vix_last
            vix["vix_30avg"] = vix_30
            vix["vix_elev"] = (vix_last > vix_30)
        else:
            vix["vix_last"] = None
            vix["vix_30avg"] = None
            vix["vix_elev"] = None
    except Exception:
        vix = {
            "asof": None, "last": None, "dma200": None, "trend": None,
            "r1m": None, "r3m": None, "high12": None, "drawdown": None,
            "vix_last": None, "vix_30avg": None, "vix_elev": None,
            "source": "Yahoo Finance (yfinance)"
        }

    # Funds
    us = {t: compute_metrics(fetch_close_series(t)) for t in US_ENGINE}
    intl = compute_metrics(fetch_close_series(INTERNATIONAL))
    defense = compute_metrics(fetch_close_series(DEFENSE))
    gold = compute_metrics(fetch_close_series(GOLD))
    opp = {t: compute_metrics(fetch_close_series(t)) for t in OPPORTUNISTIC}

    return {"SPY": spy, "QQQ": qqq, "VIX": vix, "US": us, "INTL": intl, "DEF": defense, "GOLD": gold, "OPP": opp}

# -------------------------------
# FRESHNESS CONTROL (optional)
# -------------------------------
def enforce_freshness(data: dict):
    target = data["SPY"]["asof"]
    required = dict(data["US"])
    required[INTERNATIONAL] = data["INTL"]
    required[DEFENSE] = data["DEF"]
    required[GOLD] = data["GOLD"]

    stale = [(t, required[t]["asof"]) for t in required if required[t]["asof"] != target]
    if stale:
        print(f"SPY as-of (target): {target}")
        print("STALE tickers (ticker, as-of):")
        for t, a in stale:
            print(f"  {t}: {a}")
        raise RuntimeError("STALE DATA")

def gather_with_freshness() -> dict:
    if FRESH_RETRY_MINUTES <= 0:
        data = gather_all_data()
        if REQUIRE_FRESH:
            enforce_freshness(data)
        return data

    deadline = time.time() + (FRESH_RETRY_MINUTES * 60)
    last_err = None
    while True:
        try:
            data = gather_all_data()
            if REQUIRE_FRESH:
                enforce_freshness(data)
            return data
        except Exception as e:
            last_err = e
            if ("STALE" in str(e).upper()) and (time.time() < deadline):
                print(f"Still stale. Retrying in {FRESH_RETRY_EVERY_MINUTES} minutes...")
                time.sleep(FRESH_RETRY_EVERY_MINUTES * 60)
                continue
            raise last_err

# -------------------------------
# PART 7: ELIGIBILITY RULES (YES/NO ONLY)
# -------------------------------
def apply_eligibility(data: dict) -> dict:
    spy = data["SPY"]; qqq = data["QQQ"]; vix = data["VIX"]
    spy_trend = _require("trend/200DMA", spy["trend"], "SPY")
    qqq_trend = _require("trend/200DMA", qqq["trend"], "QQQ")

    risk_on = (spy_trend == "ABOVE") and (qqq_trend == "ABOVE")

    # A) US funds
    us_results = {}
    for t, m in data["US"].items():
        mp = risk_on
        mom = _require("1M return", m["r1m"], t) > 0.0
        tr = _require("trend/200DMA", m["trend"], t) == "ABOVE"
        final = mp and mom and tr
        us_results[t] = {"market_permission": mp, "momentum": mom, "trend": tr, "eligible": final}

    # B) International
    intl = data["INTL"]
    mp_i = risk_on
    mom_i = (_require("1M return", intl["r1m"], INTERNATIONAL) > 0.0) and (_require("3M return", intl["r3m"], INTERNATIONAL) > 0.0)
    tr_i = _require("trend/200DMA", intl["trend"], INTERNATIONAL) == "ABOVE"
    intl_ok = mp_i and mom_i and tr_i

    # C) Defense
    d = data["DEF"]
    mp_d = risk_on
    mom_d = _require("1M return", d["r1m"], DEFENSE) > 0.0
    tr_d = _require("trend/200DMA", d["trend"], DEFENSE) == "ABOVE"
    def_ok = mp_d and mom_d and tr_d

    # D) Gold / Crisis
    g = data["GOLD"]
    mp_g = (spy_trend == "ABOVE")
    mom_g = _require("1M return", g["r1m"], GOLD) > 0.0
    tr_g = _require("trend/200DMA", g["trend"], GOLD) == "ABOVE"
    vix_confirm = None
    if vix.get("vix_elev", None) is not None:
        vix_confirm = bool(vix["vix_elev"])
    gold_ok = (mp_g and mom_g and tr_g) and (vix_confirm if vix_confirm is not None else True)

    # E) Opportunistic
    opp_results = {}
    for t, m in data["OPP"].items():
        mp_o = risk_on
        dd = m["drawdown"]
        val = (dd is not None) and (dd <= -20.0)  # drawdown >=20% from high
        final = mp_o and val
        opp_results[t] = {"market_permission": mp_o, "valuation": val, "eligible": final}

    return {
        "risk_on": risk_on,
        "spy_trend": spy_trend,
        "qqq_trend": qqq_trend,
        "us": us_results,
        "intl": {"market_permission": mp_i, "momentum_valid": mom_i, "trend_valid": tr_i, "eligible": intl_ok},
        "def": {"market_permission": mp_d, "momentum": mom_d, "trend": tr_d, "eligible": def_ok},
        "gold": {"market_permission": mp_g, "momentum": mom_g, "trend": tr_g, "vix_confirm": vix_confirm, "eligible": gold_ok},
        "opp": opp_results,
    }

# -------------------------------
# PART 9: MONDAY TRADE TICKET (STRICT RULES)
# -------------------------------
def select_engine(data: dict, elig: dict) -> str:
    if not elig["risk_on"]:
        return "FCNTX"

    # Determine top eligible US fund by 1M return
    eligible_us = {t: data["US"][t]["r1m"] for t, r in elig["us"].items() if r["eligible"]}
    top_us_ticker = None
    top_us_r1m = None
    if eligible_us:
        top_us_ticker = max(eligible_us, key=eligible_us.get)
        top_us_r1m = float(eligible_us[top_us_ticker])

    intl_ok = elig["intl"]["eligible"]
    intl_1m = float(data["INTL"]["r1m"])

    # Rule: Intl eligible AND 1M exceeds top eligible US by >=5.00% -> use FHKCX
    if intl_ok and (top_us_r1m is not None) and ((intl_1m - top_us_r1m) >= 5.0):
        return INTERNATIONAL

    # Otherwise: top eligible US by 1M
    if top_us_ticker is not None:
        return top_us_ticker

    # If none eligible (still risk-on): default to FCNTX
    return "FCNTX"

def build_ticket(data: dict, elig: dict) -> list:
    # Base
    ticket = dict(BASE_BUYS)

    # Engine
    engine = select_engine(data, elig)
    ticket[engine] = ticket.get(engine, 0) + ENGINE_BUDGET

    # Flex to FXAIX initially
    ticket["FXAIX"] = ticket.get("FXAIX", 0) + (BUDGET_TOTAL - sum(ticket.values()))

    # Optional sleeves (cap 2)
    sleeves = []
    if elig["def"]["eligible"]:
        sleeves.append(DEFENSE)
    if elig["gold"]["eligible"]:
        sleeves.append(GOLD)

    # Opportunistic: pick ONE eligible if any (deterministic: most negative drawdown)
    opp_eligible = [t for t, r in elig["opp"].items() if r["eligible"]]
    opp_pick = None
    if opp_eligible:
        opp_pick = min(opp_eligible, key=lambda t: data["OPP"][t]["drawdown"])
        sleeves.append(opp_pick)

    sleeves = sleeves[:MAX_SLEEVES]

    for s in sleeves:
        ticket["FXAIX"] -= SLEEVE_AMOUNT
        if ticket["FXAIX"] < 0:
            raise RuntimeError("FXAIX allocation went negative after sleeves.")
        ticket[s] = ticket.get(s, 0) + SLEEVE_AMOUNT

    # Validate total
    total = sum(ticket.values())
    if total != BUDGET_TOTAL:
        raise RuntimeError(f"Ticket total != {BUDGET_TOTAL} (got {total})")

    # Return stable ordered list
    return [(k, ticket[k]) for k in sorted(ticket.keys())]

# -------------------------------
# PRINT FORMATS (STRICT)
# -------------------------------
def print_part1(data: dict):
    spy = data["SPY"]; qqq = data["QQQ"]; vix = data["VIX"]
    print("PART 1 — MARKET REGIME")
    print(f"SPY Last: {spy['last']:.2f} | 200DMA: {spy['dma200']:.2f} | Status: {spy['trend']} | Source: {spy['source']} | As-of: {spy['asof']}")
    print(f"QQQ Last: {qqq['last']:.2f} | 200DMA: {qqq['dma200']:.2f} | Status: {qqq['trend']} | Source: {qqq['source']} | As-of: {qqq['asof']}")
    print(f"Risk-On Permission: {_yesno((spy['trend']=='ABOVE') and (qqq['trend']=='ABOVE'))}")
    if vix.get("vix_last", None) is not None and vix.get("vix_30avg", None) is not None:
        print(f"VIX Last Close: {vix['vix_last']:.2f} | VIX 30-day Avg: {vix['vix_30avg']:.2f} | VIX Elevated: {_yesno(bool(vix['vix_elev']))} | Source: {vix['source']} | As-of: {vix['asof']}")
    print()

def print_part2(data: dict):
    print("PART 2 — US ENGINE UNIVERSE")
    for t in US_ENGINE:
        m = data["US"][t]
        print(f"{t} NAV: {m['last']:.4f} | 200DMA: {m['dma200']:.4f} | Trend: {m['trend']} | 1M Return: {m['r1m']:.2f}% | Source: {m['source']} | As-of: {m['asof']}")
    print()

def print_part3(data: dict):
    print("PART 3 — INTERNATIONAL CHECK")
    m = data["INTL"]
    print(f"{INTERNATIONAL} 1M: {m['r1m']:.2f}% | 3M: {m['r3m']:.2f}% | NAV: {m['last']:.4f} | 200DMA: {m['dma200']:.4f} | Trend: {m['trend']} | Source: {m['source']} | As-of: {m['asof']}")
    print()

def print_part4(data: dict):
    print("PART 4 — DEFENSE CHECK")
    m = data["DEF"]
    print(f"{DEFENSE} 1M: {m['r1m']:.2f}% | NAV: {m['last']:.4f} | 200DMA: {m['dma200']:.4f} | Trend: {m['trend']} | Source: {m['source']} | As-of: {m['asof']}")
    print()

def print_part5(data: dict):
    print("PART 5 — CRISIS / GOLD CHECK")
    m = data["GOLD"]
    print(f"{GOLD} 1M: {m['r1m']:.2f}% | NAV: {m['last']:.4f} | 200DMA: {m['dma200']:.4f} | Trend: {m['trend']} | Source: {m['source']} | As-of: {m['asof']}")
    print()

def print_part6(data: dict):
    print("PART 6 — OPPORTUNISTIC DRAWDOWN CHECK")
    for t in OPPORTUNISTIC:
        m = data["OPP"][t]
        dd = m["drawdown"]
        dd_s = f"{dd:.2f}%" if dd is not None else "NA"
        h12 = m["high12"]
        h12_s = f"{h12:.4f}" if h12 is not None else "NA"
        print(f"{t} NAV: {m['last']:.4f} | 12M High: {h12_s} | Drawdown: {dd_s} | 1M: {m['r1m']:.2f}% | Source: {m['source']} | As-of: {m['asof']}")
    print()

def print_part7(elig: dict):
    print("PART 7 — ELIGIBILITY RULES (YES/NO ONLY)")
    # A) US
    for t in US_ENGINE:
        r = elig["us"][t]
        print(f"{t} | Market Permission: {_yesno(r['market_permission'])} | Momentum: {_yesno(r['momentum'])} | Trend: {_yesno(r['trend'])} | FINAL US ROTATION ELIGIBLE: {_yesno(r['eligible'])}")
    # B) Intl
    i = elig["intl"]
    print(f"{INTERNATIONAL} | Market Permission: {_yesno(i['market_permission'])} | Momentum Valid: {_yesno(i['momentum_valid'])} | Trend Valid: {_yesno(i['trend_valid'])} | FINAL INTERNATIONAL ELIGIBLE: {_yesno(i['eligible'])}")
    # C) Defense
    d = elig["def"]
    print(f"{DEFENSE} | Market Permission: {_yesno(d['market_permission'])} | Momentum: {_yesno(d['momentum'])} | Trend: {_yesno(d['trend'])} | FINAL DEFENSE ELIGIBLE: {_yesno(d['eligible'])}")
    # D) Gold
    g = elig["gold"]
    vc = g["vix_confirm"]
    vc_s = "NA" if vc is None else _yesno(vc)
    print(f"{GOLD} | Market Permission: {_yesno(g['market_permission'])} | Momentum: {_yesno(g['momentum'])} | Trend: {_yesno(g['trend'])} | VIX Confirm: {vc_s} | FINAL GOLD ELIGIBLE: {_yesno(g['eligible'])}")
    # E) Opportunistic
    for t in OPPORTUNISTIC:
        r = elig["opp"][t]
        print(f"{t} | Market Permission: {_yesno(r['market_permission'])} | Valuation (Drawdown>=20%): {_yesno(r['valuation'])} | FINAL OPPORTUNISTIC ELIGIBLE: {_yesno(r['eligible'])}")
    print()

def print_part8(data: dict, elig: dict):
    print("PART 8 — OUTPUT SUMMARY (NO COMMENTARY)")
    print(f"Market Regime: SPY={elig['spy_trend']} | QQQ={elig['qqq_trend']} | Risk-On Permission={_yesno(elig['risk_on'])}")
    for t in US_ENGINE:
        m = data["US"][t]
        print(f"{t} — 1M: {m['r1m']:.2f}% | Trend: {m['trend']} | Eligible: {_yesno(elig['us'][t]['eligible'])}")
    m = data["INTL"]
    print(f"{INTERNATIONAL} — 1M: {m['r1m']:.2f}% | 3M: {m['r3m']:.2f}% | Trend: {m['trend']} | Eligible: {_yesno(elig['intl']['eligible'])}")
    m = data["DEF"]
    print(f"{DEFENSE} — 1M: {m['r1m']:.2f}% | Trend: {m['trend']} | Eligible: {_yesno(elig['def']['eligible'])}")
    m = data["GOLD"]
    print(f"{GOLD} — 1M: {m['r1m']:.2f}% | Trend: {m['trend']} | Eligible: {_yesno(elig['gold']['eligible'])}")
    vix = data["VIX"]
    if vix.get("vix_elev", None) is not None:
        print(f"(Optional) VIX Elevated: {_yesno(bool(vix['vix_elev']))}")
    for t in OPPORTUNISTIC:
        dd = data["OPP"][t]["drawdown"]
        dd_s = f"{dd:.2f}%" if dd is not None else "NA"
        print(f"{t} — Drawdown: {dd_s} | Eligible: {_yesno(elig['opp'][t]['eligible'])}")
    print()

def print_part9(ticket: list):
    print("PART 9 — MONDAY TRADE TICKET (STRICT RULES)")
    for t, amt in ticket:
        print(f"{t}: ${amt}")

# -------------------------------
# TOP-LEVEL RUNNERS
# -------------------------------
def RUN_FULL():
    data = gather_with_freshness()
    elig = apply_eligibility(data)

    print_part1(data)
    print("AS-OF DATES (validation)")
    print(f"SPY: {data['SPY']['asof']} | QQQ: {data['QQQ']['asof']}")
    for t in US_ENGINE:
        print(f"{t}: {data['US'][t]['asof']}")
    print(f"{INTERNATIONAL}: {data['INTL']['asof']}")
    print(f"{DEFENSE}: {data['DEF']['asof']}")
    print(f"{GOLD}: {data['GOLD']['asof']}")
    for t in OPPORTUNISTIC:
        print(f"{t}: {data['OPP'][t]['asof']}")
    print()

    print_part2(data)
    print_part3(data)
    print_part4(data)
    print_part5(data)
    print_part6(data)
    print_part7(elig)
    print_part8(data, elig)

    ticket = build_ticket(data, elig)
    print_part9(ticket)

def RUN_TICKET():
    data = gather_with_freshness()
    elig = apply_eligibility(data)
    ticket = build_ticket(data, elig)
    # Strict output: final Monday buy list only
    for t, amt in ticket:
        print(f"{t}: ${amt}")

In [ ]:
RUN_FULL()